#### AMPEL intro

AMPEL is software framework designed for processing heterogeneous streamed data. 

AMPEL was not developed to provide a specific scientific resource, but rather an environment where it is easy to ensure that a scientific program fulfills the strict requirement of the next generation real-time experiments: efficient and powerful analysis, where provenance and reproducibiltiy is paramount. In particular, to guarantee the last point requires algorithms (which make real-time deicsions) be separated from infrastructure (which will likely evolve with time and project phase).

An AMPEL _user_ constructs a configuration file which describes every step of how an incoming alert stream should be processed. This can be broken down into selecting which _units_ should be executed, and which _parameters_ each of these should be provided. An AMPEL _live instance_ executes these units, based on the input data, as requested and stores all intermediate and final data in a databse. 

Provenance/reproducibility is ensured through multiple layers. First, each live instance is run from a container which can be retrieved later and together with a data archive replay the full stream. Second, AMPEL contains an extensive set of logs and a transient-specific _Journal_ which details all related actions/decisions. Finally, each unit and channel configuration file is drawn from a specific (tagged) github version. 

The series of notebooks provided here gradually builds toward a sample full configration.

#### Sample science case

Each AMPEl _channel_ is designed with a science goal (or "hypothesis/test") in mind. A much discussed current topic is the origin of the extragalactic neutrino flux observed e.g. by IceCube, with one of the potential sources being supernovae interacting with circumstellar material (SNIIn). We here wish to investigate whether a particular subtype of these, SN2009ip-like SNe with recent previous outbursts, are regularly found within the uncertainty region of neutrino alerts. 

The steps for this science program would be: Identify transients with optical lightcurves compatible with SN2009ip AND which coincide with neutrino alerts. For such targets, obtain follow-up spectroscopy to confirm classification (i.e. an external reaction). 

#### This notebook - Tutorial 4

This notebook reproduces the results of Tutorial 3, but through using external configuration files. These work as "recipes" for a scientific program. Subsequent analysis verisons can be expressed through changes to these, and they can be conveniently distributed e.g. in publications. Versions of the same sample channel is provided both as `SAMPLE_CHANNEL.yml` in the `conf/ampel-contrib/sample/channel` dir and as two processes in the `conf/ampel-contrib/sample/process` dir. The content of `SAMPLE_CHANNEL.yml` is included the cell below. The generation of the core `ampel_config.yml` file included all of these.

At this stage the channel is ready to be included in a live AMPEL instance. This can either be used to process a large set of archive data, or for processing a real-time data stream. Simultaneously, the channel configuration and the unit algorithms serves to provide a full, referencable description of the science content of the channel. 

As in Tutorial 2, this notebook thus assumes a mongod instance to be running and accessible through 27017. (The port can be changed through the mongo key of the ampel_config.yml file).

###### Content of SAMPLE_CHANNEL.yml

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext AmpelLogger T2Processor T3Processor ChannelModel AlertProcessor TarAlertLoader ChannelModel AbsAlertFilter ProcessModel AbsProcessorUnit DefaultProcessController

In [ ]:
AMPEL_CONF = "../../ampel_config.yml"
ALERT_ARCHIVE = '../sample_data/ztfpub_200917_pruned.tar.gz'

In [ ]:
# The operation context is created based on a setup configuration file.
# db_prefix sets the DB name to use
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTutorial",
    purge_db = True,
)

First the t0 selection/filter process will be run, as gathered from the 'process' folder:

In [ ]:
process_name = "process.t0.sample_t0_process"

In [ ]:
# A process model is determined and executed by a processor
pm = ProcessModel(**ctx.config.get(process_name))
ctx.loader.new_admin_unit(unit_model = pm.processor, context = ctx, process_name = pm.name).run()

Outcome is the same as previously. This process also created "tickets" for T2 calculations to do. These will now be executed:

In [ ]:
process_name = "process.t2.DefaultT2Process"

In [ ]:
pm = ProcessModel(**ctx.config.get(process_name))
ctx.loader.new_admin_unit(unit_model = pm.processor, context = ctx, process_name = pm.name).run()

In [ ]:
process_name = "process.t3.sample_t3_process"

In [ ]:
pm = ProcessModel(**ctx.config.get(process_name))
ctx.loader.new_admin_unit(unit_model = pm.processor, context = ctx, process_name = pm.name).run()